In [ ]:
import mysql.connector
import pandas as pd
import time
import itertools
import math

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline

import plotly.express as px
import datetime
from IPython.display import clear_output

import plotly.offline as py
import plotly.graph_objs as go
from plotly.subplots import make_subplots

py.init_notebook_mode()

import re
import nltk
from nltk.probability import FreqDist
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords


In [ ]:
while True:
    clear_output()
    db_connection = mysql.connector.connect(
        host="localhost",
        user="root",
        passwd="XXXXX",
        database="TWITTERDB",
        charset = 'utf8'
    )
    
    timenow = (datetime.datetime.utcnow() - datetime.timedelta(hours=0, minutes=10)).strftime('%Y-%m-%d %H:%M:%S')
    query = "SELECT id_str, text, created_at, rating, user_location FROM {} WHERE created_at >= '{}' ".format("TWITTER", timenow)
    df = pd.read_sql(query, con=db_connection)

    df['created_at'] = pd.to_datetime(df['created_at'])

    result = df.groupby([pd.Grouper(key='created_at', freq='2s'), 'rating']).count().unstack(fill_value=0).stack().reset_index()
    
    result = result.rename(columns={"id_str": "Num of '{}' mentions".format("@amazon"), "created_at":"Time in UTC"})
    
    fig = px.line(result, x='Time in UTC', y="Num of '{}' mentions".format("@amazon"), color='rating')
    fi.show()
    
    content = ' '.join(df["text"])
    content = re.sub(r"http\S+", "", content)
    content = content.replace('RT ', ' ').replace('&amp;', 'and')
    content = re.sub('[^A-Za-z0-9]+', ' ', content)
    content = content.lower()

    tokenized_word = word_tokenize(content)
    stop_words=set(stopwords.words("english"))
    filtered_sent=[]
    for w in tokenized_word:
        if w not in stop_words:
            filtered_sent.append(w)

    fdist = FreqDist(filtered_sent)
    fd = pd.DataFrame(fdist.most_common(25), columns = ["Word","Frequency"]).drop([0]).reindex()

    fig = px.bar(fd, x="Word", y="Frequency")
    fig.update_traces(marker_color='rgb(240,128,128)', marker_line_color='rgb(8,48,107)', marker_line_width=1.5, opacity=0.8)
    fig.show()
    
    
    labels = df['rating'].value_counts().index
    values = df2['rating'].value_counts().values
    
    trace = go.Pie(labels=labels, values=values)
    
    fig = py.iplot([trace], filename='basic_pie_chart')
    fig.show()
    
    


    time.sleep(60)